In [ ]:
import json
import re
import os
import requests
from googleapiclient.discovery import build
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from IPython.display import HTML

SERVICE_ACCOUNT_FILE = 'keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
creds = None 
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
SAMPLE_SPREADSHEET_ID = '1W...9k' # Not provided for privacy reasons
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets().values()


In [ ]:
class ScrapeData():

    def read_from_link(self, link, replace_list=['\n','\t','\r','\u200d','\xa0']):
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        data = []
        if (link.find('esamaad') >= 0):
            find = soup.find_all('div', attrs={'style':'text-align: justify'})
            if(not find):
                find = soup.find_all('p', attrs={'style':'text-align: justify'})
                if(not find):
                    find = soup.find_all('p')
        elif(link.find('vikaspedia') >= 0):
            find = soup.find_all('div', attrs={'id':'texttospeak'})
        else:
            find = soup.find_all('p')
        for i in range(len(find)):
            text = find[i].get_text()
            text = re.sub("[a-zA-Z-|©&@:/^%`'$#•ｾ＠!◌<>{}_–‘’“”=~;Ø(,+*)\[\]]*", "", text)
            for j in range(len(replace_list)):
                text = text.replace(replace_list[j],'')
            text = re.sub("[.]+", ".", text)
            text = re.sub(' +', ' ', text)
            text = text.strip()
            if len(text)>2:
                data.append(text)

        return data


    def read_page(self, link=None):

        if link:
            text = self.read_from_link(link)
            return(text)



def link_data(url):
    data = []
    if(url.lower().endswith('.pdf')):
        return data
    try:
      data = scrape_tool.read_page(link=url)
      data = list(set(data))
    except:
      data += []

    return data


In [ ]:
def fetchData(i, types, Ranges):
    
    folder = types[i]
    
    if not os.path.exists("jsonOutputs"):
        os.mkdir("jsonOutputs")
    if not os.path.exists(f"jsonOutputs/{folder}"):
        os.mkdir(f"jsonOutputs/{folder}")
    
    SAMPLE_RANGE_NAME = f'{folder.upper()}!{Ranges[i]}'
    rows = sheet.get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME, valueRenderOption='FORMULA').execute().get('values', [])
    
    topic = ''
    data = []

    contents = []
    for r in rows:
        if(len(r)>0 and r[0]):
            if(topic and contents):
                title = ''.join(e for e in topic if e.isalnum())
                with open(f'jsonOutputs/{folder}/{title}.json', 'w+', encoding='utf-8') as f:
                    json.dump(contents, f, ensure_ascii=False, indent=4)
            
            topic = re.findall('"([^"]*)"',r[0])
            contents = []
            if(topic):
                topic = topic[1]
            if(not (topic and topic.strip())):
                topic = r[0]

        sub_topic = ''
        if(len(r)>1 and r[1]):
            sub_topic = re.findall('"([^"]*)"',r[1])
            if(sub_topic):
                sub_topic = sub_topic[1]
            if(not (sub_topic and sub_topic.strip())):
                sub_topic = r[1]

        if(len(r)>2 and r[2]):

            data = []

            for c in r[2:]:
                link = re.findall('"([^"]*)"',c)
                if(link):
                    link = link[0]
                    if(link):
                        linkdata = link_data(link)
                        if(linkdata):
                            linkdata = {'url': link, 'data': linkdata}
                            data.append(linkdata.copy())

            if(data):
                out = {'topic': topic, 'sub_topic': sub_topic, 'data': data}
                contents.append(out.copy())


In [ ]:
scrape_tool = ScrapeData()
types = ['agriculture', 'finance', 'general'] 
Ranges = ['C3:M530', 'C3:M530', 'C3:O1120']

In [ ]:
i = 0
# fetchData(i, types, Ranges)

In [ ]:
i = 1
# fetchData(i, types, Ranges)

In [ ]:
i = 2
# fetchData(i, types, Ranges)